## Apriori & FP tree

In [ ]:
!pip install pip --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 10.8 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2


In [ ]:
!pip install mlxtend==0.18

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 9.3 MB/s eta 0:00:00
  Attempting uninstall: mlxtend
    Found existing installation: mlxtend 0.23.4
    Uninstalling mlxtend-0.23.4:
      Successfully uninstalled mlxtend-0.23.4


In [ ]:
import mlxtend
mlxtend.__version__

'0.18.0'

In [ ]:
from mlxtend.frequent_patterns import fpgrowth
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [ ]:
import pandas as pd
import os

## Dataset import 및 확인

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

basicpath = '/content/drive/MyDrive/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
path = basicpath + 'CBNUDatascience_colab/2-3주차'
file = 'BreadBasket_DMS.csv'
data = pd.read_csv(os.path.join(path, file), index_col = None)

In [ ]:
data.head(10)

,Date,Time,Transaction,Item
0,2016.10.30,9:58:11,1,Bread
1,2016.10.30,10:05:34,2,Scandinavian
2,2016.10.30,10:05:34,2,Scandinavian
3,2016.10.30,10:07:57,3,Hot chocolate
4,2016.10.30,10:07:57,3,Jam
5,2016.10.30,10:07:57,3,Cookies
6,2016.10.30,10:08:41,4,Muffin
7,2016.10.30,10:13:03,5,Coffee
8,2016.10.30,10:13:03,5,Pastry
9,2016.10.30,10:13:03,5,Bread


In [ ]:
data.shape

(21293, 4)

## 과제1-1 Data Cleaning


위 데이터를 가공하여, transaction과 item이 column이 되는  
다음과 같은 형태의 새로운 DataFrame을 만들어 변수 data에 저장하시오.

 transaction | item
:-------------:|:------:
      0      |   7  
      0      |   14
      1      |   9  
      2      |   18
...  


In [ ]:
# 1. 대문자 -> 소문자 변환 (str.lower() 활용)
data['Item'] = data['Item'].str.lower()

In [ ]:
# 2. None value 제거 (drop 활용)
data = data.drop(data[data['Item']=='none'].index)

### Data 간단 분석_item당 등장횟수

In [ ]:
data

,Date,Time,Transaction,Item
0,2016.10.30,9:58:11,1,bread
1,2016.10.30,10:05:34,2,scandinavian
2,2016.10.30,10:05:34,2,scandinavian
3,2016.10.30,10:07:57,3,hot chocolate
4,2016.10.30,10:07:57,3,jam
...,...,...,...,...
21288,2017.4.9,14:32:58,9682,coffee
21289,2017.4.9,14:32:58,9682,tea
21290,2017.4.9,14:57:06,9683,coffee
21291,2017.4.9,14:57:06,9683,pastry


In [ ]:
len(data['Item'].unique())

94

In [ ]:
top10_items = data['Item'].value_counts().head(10)
top10_items

,count
Item,
coffee,5471
bread,3325
tea,1435
cake,1025
pastry,856
sandwich,771
medialuna,616
hot chocolate,590
cookies,540


In [ ]:
len(data['Transaction'].unique())

9465

## 과제 1-2 one-hot-encoding vector 만들기
위 data를 가공하여 one-hot-encoding 형태의 데이터를 만들고  
이를 hot_encoded_data 변수에 저장하시오  
이때 one-hot-encoding 형태의 column은 item, row는 transaction이어야 함  

In [ ]:
#답변
hot_encoded_data = data.groupby(['Transaction','Item'])['Item'].count().unstack().fillna(0)
hot_encoded_data

Item,adjustment,afternoon with the baker,alfajores,argentina night,art tray,bacon,baguette,bakewell,bare popcorn,basket,...,the bart,the nomad,tiffin,toast,truffles,tshirt,valentine's card,vegan feast,vegan mincepie,victorian sponge
Transaction,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9680,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9681,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
9682,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
#아래의 코드를 설명하시오:
# 전 과정에서 각 거래별 아이템이 몇개씩 구매되었는지 표시하므로 빈도를 표시한다.
# 빈도수를 0과 1로 이루어진 one hot vector로 바꾸어야 함으로 1이상인 경우 1을 삽입하는 코드가 다음과 같다.
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

hot_encoded_data = hot_encoded_data.applymap(encode_units)
hot_encoded_data

<ipython-input-41-7febe84ce6a8>:8: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  hot_encoded_data = hot_encoded_data.applymap(encode_units)


Item,adjustment,afternoon with the baker,alfajores,argentina night,art tray,bacon,baguette,bakewell,bare popcorn,basket,...,the bart,the nomad,tiffin,toast,truffles,tshirt,valentine's card,vegan feast,vegan mincepie,victorian sponge
Transaction,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9680,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9681,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
9682,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## 과제 1-3
mlxtend의 apriori & fp-growth 라이브러리를 활용하여 위 데이터에서 frequent itemset을 추출하시오.  
이때, min_support는 0.02로 하시오.  
association_rules 라이브러리를 활용하여 추출한 frequent itemset에서 association rule을 만드시오.

In [ ]:
#답변 apriori
frequent_itemsets = apriori(hot_encoded_data, 0.02, use_colnames=True)

In [ ]:
rules = association_rules(frequent_itemsets, metric='lift', min_threshold=1)

In [ ]:
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(bread),(pastry),0.327205,0.086107,0.029160,0.089119,1.034977,0.000985,1.003306
1,(pastry),(bread),0.086107,0.327205,0.029160,0.338650,1.034977,0.000985,1.017305
2,(coffee),(cake),0.478394,0.103856,0.054728,0.114399,1.101515,0.005044,1.011905
3,(cake),(coffee),0.103856,0.478394,0.054728,0.526958,1.101515,0.005044,1.102664
4,(cake),(tea),0.103856,0.142631,0.023772,0.228891,1.604781,0.008959,1.111865
5,(tea),(cake),0.142631,0.103856,0.023772,0.166667,1.604781,0.008959,1.075372
6,(cookies),(coffee),0.054411,0.478394,0.028209,0.518447,1.083723,0.002179,1.083174
7,(coffee),(cookies),0.478394,0.054411,0.028209,0.058966,1.083723,0.002179,1.004841
8,(hot chocolate),(coffee),0.058320,0.478394,0.029583,0.507246,1.060311,0.001683,1.058553
9,(coffee),(hot chocolate),0.478394,0.058320,0.029583,0.061837,1.060311,0.001683,1.003749


In [ ]:
#답변
frequent_itemsets = fpgrowth(hot_encoded_data, 0.02, use_colnames=True)

In [ ]:
rules = association_rules(frequent_itemsets, metric='lift', min_threshold=1)

In [ ]:
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(hot chocolate),(coffee),0.058320,0.478394,0.029583,0.507246,1.060311,0.001683,1.058553
1,(coffee),(hot chocolate),0.478394,0.058320,0.029583,0.061837,1.060311,0.001683,1.003749
2,(cookies),(coffee),0.054411,0.478394,0.028209,0.518447,1.083723,0.002179,1.083174
3,(coffee),(cookies),0.478394,0.054411,0.028209,0.058966,1.083723,0.002179,1.004841
4,(coffee),(pastry),0.478394,0.086107,0.047544,0.099382,1.154168,0.006351,1.014740
5,(pastry),(coffee),0.086107,0.478394,0.047544,0.552147,1.154168,0.006351,1.164682
6,(bread),(pastry),0.327205,0.086107,0.029160,0.089119,1.034977,0.000985,1.003306
7,(pastry),(bread),0.086107,0.327205,0.029160,0.338650,1.034977,0.000985,1.017305
8,(coffee),(medialuna),0.478394,0.061807,0.035182,0.073542,1.189878,0.005614,1.012667
9,(medialuna),(coffee),0.061807,0.478394,0.035182,0.569231,1.189878,0.005614,1.210871
